## Ezyme-Subtrate Kinetics Rule Creator


This notebook uses the Boolean_rules_creator tool available to generate
rules for a boolean network describing Michaelis-Menten enzyme-substrate kinetics. For more information on the tool and installation instructions, view the GitHub page at https://github.com/LoLab-VU/Boolean_rules_creator .

>More details on the algorithm and a description of the model are available in the preprint at https://www.biorxiv.org/content/10.1101/2020.12.15.422874v1.full (Prugger et. al. 2020).

### Importing libraries


In [1]:
from rule_creator import creating_rules
import json
import sympy

### Loading the input file

For the MM model, there is only one possibly steady state outcome, so the Boolean ruleset can be generation without optimization. The list  In this example, only the rule_creator function is used to recreate the forward, backwards, and expert-guided rulesets.

The input file for this model is 'MM_steady_states.json', which contains the possible initial boolean states for the four species: Enzyme (EN), Substrate (S), Enzyme-Substrate complex (ES), and Product (P). For this script, these states are always referenced in this order.

In [2]:
fn = 'ES_steady_states.json'
symbols = ['EN', 'S', 'ES', 'P']

The input file contains pairs of intial and steady states followed by the frequency
with which the intial state leads to that final state. 

In [3]:
with open(fn) as fs:
    data = json.load(fs)
    fs.close()
print(data.items())

dict_items([('[0, 0, 0, 0]', [[[0, 0, 0, 0], 100]]), ('[0, 0, 0, 1]', [[[0, 0, 0, 1], 100]]), ('[0, 0, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[0, 0, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[0, 1, 0, 0]', [[[0, 1, 0, 0], 100]]), ('[0, 1, 0, 1]', [[[0, 1, 0, 1], 100]]), ('[0, 1, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[0, 1, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 0, 0]', [[[1, 0, 0, 0], 100]]), ('[1, 0, 0, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 0, 1, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 0, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 0, 1]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 1, 0]', [[[1, 0, 0, 1], 100]]), ('[1, 1, 1, 1]', [[[1, 0, 0, 1], 100]])])


### Generating network rules

The function creating_rules will take these input states and generate a list of rules which contain
the possible transitions to describe the boolean network. For this model, the first three states, as seen below, can only progress to their respective steady states due to the lack of reactive species. For all other intial states the only possible steady state is [1,0,0,1].

In [4]:
str_rules, simple_rulelist, fs_cpp_name = creating_rules(fn,symbols,1)

	
data:  {'[0, 0, 0, 0]': [[[0, 0, 0, 0], 100]], '[0, 0, 0, 1]': [[[0, 0, 0, 1], 100]], '[0, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[0, 1, 0, 0]': [[[0, 1, 0, 0], 100]], '[0, 1, 0, 1]': [[[0, 1, 0, 1], 100]], '[0, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[0, 1, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 0, 0]': [[[1, 0, 0, 0], 100]], '[1, 0, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 0, 1, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 0, 1]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 0]': [[[1, 0, 0, 1], 100]], '[1, 1, 1, 1]': [[[1, 0, 0, 1], 100]]}
	
steady state: [0, 0, 0, 0]
	(0, 0, 0, 0) <- 100
	
	
steady state: [0, 0, 0, 1]
	(0, 0, 0, 1) <- 100
	
	
steady state: [1, 0, 0, 1]
	(0, 0, 1, 0) <- 100
	(0, 0, 1, 1) <- 100
	(0, 1, 1, 0) <- 100
	(0, 1, 1, 1) <- 100
	(1, 0, 0, 1) <- 100
	(1, 0, 1, 0) <- 100
	(1, 0, 1, 1) <- 100
	(1, 1, 0, 0) <- 100
	(1, 1, 0, 1) <- 100
	(1, 1, 1, 0) <- 100
	(1, 1, 1, 1) <- 100
	


Looking at the resulting rule list:

In [12]:
print('The string rules describe the full ruleset in formal logical terms, showing what states can activate that species.')
print('\nString rules:')
print(str_rules)

print('The simple rulelist show the string rules in their boolean form; this conveys the same information as above.')
print('\nSimple rulelist')
print(simple_rulelist)

The string rules describe the full ruleset in formal logical terms, showing what states can activate that species.

String rules:
1: EN* = Xor((( not EN and  not S and ES and  not P) or ( not EN and S and ES and P) or (EN and  not S and ES and P) or (EN and S and ES and  not P) or ( not EN and S and ES and  not P) or (EN and  not S and ES and  not P) or ( not EN and  not S and ES and P) or (EN and S and ES and P)), EN)
1: S* = Xor((( not EN and  not S and ES and  not P) or ( not EN and S and ES and P) or (EN and  not S and ES and P) or (EN and S and ES and  not P) or ( not EN and S and ES and  not P) or (EN and  not S and ES and  not P) or (EN and S and  not ES and P) or ( not EN and  not S and ES and P) or (EN and S and ES and P)), S)
1: ES* = Xor(((EN and  not S and ES and P) or (EN and S and  not ES and  not P) or (EN and S and ES and  not P) or (EN and S and  not ES and P) or (EN and S and ES and P)), ES)
1: P* = Xor((( not EN and  not S and ES and  not P) or ( not EN and S and ES 


With the input variable backwardspath set as 1, we generate the network with all possible backwards paths available
for transitions. The simple rulelist output describes the resulting network. The example below (from Figure 3) shows the network formed with the initial state [1,1,0,0].


![alt text](fig1b.jpg "ES-B Network")

### Generating simplified rules from manuscript

In order to simplify the rules further using Sympy, the string rules must first be converted to formal logic notation

In [141]:
str_rules_list = str_rules.split('\n')
count = 0
for n in str_rules_list:
    str_rules_list[count] = n.strip('1:')
    count +=1
sympy_ipnut = []
for k in range(4):
    sympy_input.append(str(str_rules_list[k]).replace('and','&').replace('or','|').replace('not ','~').replace('X|','Xor').split(' ='))
print(rules_formal) 

[[' EN*', ' Xor((( ~EN &  ~S & ES &  ~P) | ( ~EN & S & ES & P) | (EN &  ~S & ES & P) | (EN & S & ES &  ~P) | ( ~EN & S & ES &  ~P) | (EN &  ~S & ES &  ~P) | ( ~EN &  ~S & ES & P) | (EN & S & ES & P)), EN)'], [' S*', ' Xor((( ~EN &  ~S & ES &  ~P) | ( ~EN & S & ES & P) | (EN &  ~S & ES & P) | (EN & S & ES &  ~P) | ( ~EN & S & ES &  ~P) | (EN &  ~S & ES &  ~P) | (EN & S &  ~ES & P) | ( ~EN &  ~S & ES & P) | (EN & S & ES & P)), S)'], [' ES*', ' Xor(((EN &  ~S & ES & P) | (EN & S &  ~ES &  ~P) | (EN & S & ES &  ~P) | (EN & S &  ~ES & P) | (EN & S & ES & P)), ES)'], [' P*', ' Xor((( ~EN &  ~S & ES &  ~P) | ( ~EN & S & ES & P) | (EN &  ~S & ES & P) | (EN & S &  ~ES &  ~P) | (EN & S & ES &  ~P) | ( ~EN & S & ES &  ~P) | (EN &  ~S & ES &  ~P) | (EN & S &  ~ES & P) | ( ~EN &  ~S & ES & P) | (EN & S & ES & P)), P)']]


Using Sympy to simplify the Xor statement for each species yields the rules for the ES-B network as seen in the paper

In [154]:
print('Rules after simplifcation with Sympy')
print('EN*= ' + str(sympy.simplify(Xor((( ~EN &  ~S & ES &  ~P) | ( ~EN & S & ES & P) | (EN &  ~S & ES & P) | (EN & S & ES &  ~P) | ( ~EN & S & ES &  ~P) | (EN &  ~S & ES &  ~P) | ( ~EN &  ~S & ES & P) | (EN & S & ES & P)), EN))))
print('S*= ' + str(sympy.simplify(Xor((( ~EN &  ~S & ES &  ~P) | ( ~EN & S & ES & P) | (EN &  ~S & ES & P) | (EN & S & ES &  ~P) | ( ~EN & S & ES &  ~P) | (EN &  ~S & ES &  ~P) | (EN & S &  ~ES & P) | ( ~EN &  ~S & ES & P) | (EN & S & ES & P)), S))))
print('ES*= ' + str(sympy.simplify(Xor(((EN &  ~S & ES & P) | (EN & S &  ~ES &  ~P) | (EN & S & ES &  ~P) | (EN & S &  ~ES & P) | (EN & S & ES & P)), ES))))
print('P*= ' + str(sympy.simplify(Xor((( ~EN &  ~S & ES &  ~P) | ( ~EN & S & ES & P) | (EN &  ~S & ES & P) | (EN & S &  ~ES &  ~P) | (EN & S & ES &  ~P) | ( ~EN & S & ES &  ~P) | (EN &  ~S & ES &  ~P) | (EN & S &  ~ES & P) | ( ~EN &  ~S & ES & P) | (EN & S & ES & P)), P))))

Rules after simplifcation with Sympy
EN*= EN ^ ES
S*= S ^ (ES | (EN & P & S))
ES*= ES ^ (EN & (ES | S) & (P | S))
P*= P ^ (ES | (EN & S))
